In [38]:
# things we need for NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import random
from tensorflow import keras
from keras.models import load_model

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vivek\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [39]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [57]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

668 documents
49 classes ['CourseB.Tech', 'CourseB.TechInt', 'CourseM.Tech', 'CourseMBATech', 'CourseMCA', 'CoursePGDM', 'CoursePhD', 'accreditation', 'affirmative', 'btechandmbatechadmissions', 'generalfeestructure', 'goodbye', 'greeting', 'hours', 'integratedprograms', 'noanswer', 'offensivewords', 'opentoday', 'otherprogramsadmissions', 'postgradprograms', 'streamCivil', 'streamComputer', 'streamDataScience', 'streamElectrical', 'streamElectronicsandTelecommunications', 'streamIndustrialAutomation', 'streamInformationTechnology', 'streamMechanical', 'streamMechatronics', 'streamartificialintelligence', 'thanks', 'undergradprograms', 'whyArtificialIntelligence', 'whyB.Tech', 'whyB.TechIntegrated', 'whyCS', 'whyCivil', 'whyDataScience', 'whyEXTC', 'whyElectrical', 'whyIT', 'whyIndustrialAutomation', 'whyM.Tech', 'whyMBATech', 'whyMCA', 'whyMechanical', 'whyMechatronics', 'whyPGDM', 'whyPhD']
189 unique stemmed words ['!', "'s", '.', 'a', 'about', 'acceiv', 'accredit', 'admin', 'admit'

In [58]:
model = load_model('model_ChatBot.h5')

In [59]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [60]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.65
def classify(sentence):
    # generate probabilities from the model
    p = bow(sentence, words)
    
    d = len(p)
    f = len(documents)-2
    a = np.zeros([f, d])
    tot = np.vstack((p,a))
    
    results = model.predict(tot)[0]
    
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID, show_details=False):
    results = classify(sentence)
    print('Result:',results)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    #print(i)
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return (random.choice(i['responses']))
            results.pop(0)

In [61]:
response('hello there', '123', show_details=True)

Result: [('greeting', 1.0)]
context: 
tag: greeting


'Hey, how can I help you?'

In [62]:
response('btech admissions', '123', show_details=True)

Result: [('btechandmbatechadmissions', 0.9999988)]
tag: btechandmbatechadmissions


'Engineering aspirants interested for MBA Tech and BTech programs need to appear for NPAT. Admissions will be granted to various courses based on NPAT scores. You may follow the link for more information on NPAT - https://www.nmims.edu/docs/announcement-new-npat-pattern-for-engineering-programs.pdf'

In [63]:
response('b.tech admissions', '123', show_details=True)

Result: [('CourseB.Tech', 0.9980124)]
tag: CourseB.Tech


'Let me help you! We offer 4 year B. Tech Course for 12th+ Students at our Mumbai and Shirpur campuses.We provide B. Tech Degree in the following streams: Electrical (30 Seats), Data Science (60 Seats), - Information Technology (60+60 Seats), Computer Engineering (120+60 Seats, Civil (60 Seats), Electronics and Telecommunication (60+60 Seats), Mechatronics (60+60 Seats), Mechanical (60+60 Seats). *Seats: Mumbai + Shirpur; Minimum 70% in 12th grade required.'

In [64]:
response('b tech admissions', '123', show_details=True)

Result: [('btechandmbatechadmissions', 0.99879277)]
tag: btechandmbatechadmissions


'Engineering aspirants interested for MBA Tech and BTech programs need to appear for NPAT. Admissions will be granted to various courses based on NPAT scores. You may follow the link for more information on NPAT - https://www.nmims.edu/docs/announcement-new-npat-pattern-for-engineering-programs.pdf'

In [65]:
response('Btech admissions', '123', show_details=True)

Result: [('btechandmbatechadmissions', 0.9999988)]
tag: btechandmbatechadmissions


'Engineering aspirants interested for MBA Tech and BTech programs need to appear for NPAT. Admissions will be granted to various courses based on NPAT scores. You may follow the link for more information on NPAT - https://www.nmims.edu/docs/announcement-new-npat-pattern-for-engineering-programs.pdf'

In [66]:
response('B.Tech admissions', '123', show_details=True)

Result: [('CourseB.Tech', 0.9980124)]
tag: CourseB.Tech


'Let me help you! We offer 4 year B. Tech Course for 12th+ Students at our Mumbai and Shirpur campuses.We provide B. Tech Degree in the following streams: Electrical (30 Seats), Data Science (60 Seats), - Information Technology (60+60 Seats), Computer Engineering (120+60 Seats, Civil (60 Seats), Electronics and Telecommunication (60+60 Seats), Mechatronics (60+60 Seats), Mechanical (60+60 Seats). *Seats: Mumbai + Shirpur; Minimum 70% in 12th grade required.'

In [67]:
response('B Tech admissions', '123', show_details=True)

Result: [('btechandmbatechadmissions', 0.99879277)]
tag: btechandmbatechadmissions


'Engineering aspirants interested for MBA Tech and BTech programs need to appear for NPAT. Admissions will be granted to various courses based on NPAT scores. You may follow the link for more information on NPAT - https://www.nmims.edu/docs/announcement-new-npat-pattern-for-engineering-programs.pdf'

In [68]:
response('b-tech admissions', '123', show_details=True)

Result: [('otherprogramsadmissions', 0.8465403)]
tag: otherprogramsadmissions


'The admission details for this course are updated on our website whenever the admission process begins. Please follow this link to view all ongoing admission processes - https://engineering.nmims.edu/academics/admissions/'